In [1]:
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

map_location = torch.device('cuda:0')

In [2]:
import os
import shutil
import nip

path_to_config = '/home/msst/repo/Quantization/configs/vq/llama2_7b/trellis/llama2_trellis_128seq_1e.yaml'

config = nip.load(path_to_config)
logdir = config['logdir']
os.makedirs(logdir, exist_ok=True)
try:
    shutil.copy(path_to_config, logdir)
except shutil.SameFileError:
    pass


fp_blocks_path = config['fp_blocks_path']
q_blocks_path = config['q_blocks_path']
trained_q_blocks_path = os.path.join(config['logdir'], 'per_block_q_trained')

os.makedirs(trained_q_blocks_path, exist_ok=True)

In [3]:
trainer = qlib.HomequantTrainerPTQ(
    logdir = config['logdir'],
    training_config = config['training_params'],
    tokenizer=qlib.load_tokenizer(config['model_name']),
    rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),
	quant_classes=[qlib.SymHQLinear, qlib.HaarSymHQLinear],
	store_dtype=torch.float16
)

/tmp/ipykernel_847158/850949115.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),


Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

In [4]:
print("model_decoder.embed_tokens")
trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.embed_tokens',
        path_to_q_block = f'{q_blocks_path}/model.embed_tokens',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.embed_tokens',
        train=False,
        with_input_preparation=False,
    )

for layer_idx in range(32):
    print()
    print('layer_idx:', layer_idx)
        
    trainer.logger.block_label = f'block.{layer_idx}'
    trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block = f'{q_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.layers.{layer_idx}',
        train=True,
        with_input_preparation=True,
    )

model_decoder.embed_tokens


/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_train.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block = torch.load(path_to_fp_block, map_location=self.device


layer_idx: 0


training block (epoch 1):   0%|          | 0/64 [00:00<?, ?it/s]/home/msst/miniconda3/envs/qenv/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: quip_lib::hadamard: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:62.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
collecting activations: 1


layer_idx: 1


...

collecting activations: 100%|██████████| 16/16 [00:04<00:00,  3.53it/s]



layer_idx: 30


collecting activations: 100%|██████████| 16/16 [00:04<00:00,  3.46it/s]



layer_idx: 31


collecting activations: 100%|██████████| 16/16 [00:04<00:00,  3.49it/s]
